In [2]:
# Create a directory from going_modular script
import os
os.makedirs("going_modular")

In [4]:
%%writefile going_modular/data_setup.py
"""
Contains functionality for creating PyTorch DataLoader's for image classification data.
"""
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_worksers: int=NUM_WORKERS
):
    """
    Creates taraining a testing DataLoaders

    Takes in a training directory and testing directory path and turns them into PyTorch Datasets and then into PyTorch Dataloaders.

    Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.

    Returns:
        A tuple of (train_dataloader, test_dataloader, class_names)
        Where class_names is a list of the target classes.
        Example usage:
            train_dataloader, test_dataloader, class_names = create_dataloaders(
                                train_dir=pat/to/train_dir,
                                test_dir=path/to/test_dir,
                                batch_size=32.
                                num_workers=4
                )
    """
    #Use ImageFolder to create datasets(s)
    train_data = datasets.ImageFolder(root=train_dir,
                                    transform=transform, # a transform for the data
                                    target_transform=None) # a transform for the label/target

    test_data = datasets.ImageFolder(root=test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into Dataloaders
    train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=num_worksers, pin_memory=True)
    test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers=num_worksers, pin_memory=True)

    train_dataloader, train_dataloader
    return  train_dataloader, test_dataloader, class_names


Overwriting going_modular/data_setup.py


In [11]:
%%writefile going_modular/model_builder.py
"""
Contains pytorch model code to instantiate a TinyVGG model from CNN explainer
"""
import torch

from torch import nn

class TinyVGG(nn.Module):
    """
    Creates the TinyVGG achitecture.

    Replicates the TinyVGG architecture from the CNN explainer website in pytorch.
    See the original architecture here: https://poloclub.github.io/cnn-explainer/

    Args:
        input_shape: An integer indicating number of input channels.
        hidden_units: An integer indicating number of hidden units between layers.
        output_shape: An integer indicating number of output units.
    """
    def __init__(self, input_shape:int, hidden_units: int, output_shape:int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )


        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 30 * 30, out_features=output_shape)
        )

    def forward(self, x):
        # x = self.conv_block_1(x)
        # # print(x.shape)
        # x = self.conv_block_2(x)
        # # print(x.shape)
        # x = self.conv_block_3(x)
        # # print(x.shape)
        # x = self.classifier(x)
        # # print(x.shape)
        # return x
        return self.classifier(self.conv_block_2(self.conv_block_1(x))) # benefits from operator fusion

Overwriting going_modular/model_builder.py


In [ ]:
import torch

from going_modular import model_builder

device = "cuda" if torch.cuda.is_available() else "cpu"

# INnstantiate a model from the model_builder.py script
torch.manual_seed(42)
model_1 = model_builder.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names).to(device))

In [8]:
%%writefile going_modular/engine.py

"""
Contains function for training and testing a pytorch model
"""

from typing import Tuple, Dict, List
import torch
from timeit import default_timer as timer
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else 'cpu'

def train_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, optimizer: torch.optim.Optimizer, device=device):
    # Put the model in train mode
    model.train()

    # Setup  train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to the target device
        X, y = X.to(device), y.to(device)

        # Forward pass
        y_pred = model(X) # output model logits

        # Calculate the loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # Optimizer zero grad
        optimizer.zero_grad()

        # Loss backward
        loss.backward()

        # Optimizer.step
        optimizer.step()

        # Calculate accuracy metric
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += ((y_pred_class == y).sum().item()/len(y_pred))


    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)

    return train_loss, train_acc

def test_step(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, loss_fn: torch.nn.Module, device=device):
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on infrence mode
    with torch.inference_mode():
        # Loop through Dataloader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to the target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate the loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate the accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc




# Create a train function that takes in various parameters
def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module = torch.nn.CrossEntropyLoss(),
          epochs: int=5,
          device=device):
    # Create empty result dict
    results = {"train_loss": [], "train_acc": [], "test_loss":[], "test_acc":[]}


    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc =  train_step(model=model, dataloader=train_dataloader,loss_fn=loss_fn, optimizer=optimizer,device=device)
        test_loss, test_acc = test_step(model=model, dataloader=test_dataloader, loss_fn=loss_fn, device=device)

        # Print out
        print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc:{test_acc:.4f}")

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results



Overwriting going_modular/engine.py


# 5.1 Create a file called utils.py with utility functions

"utils" in Python is generallyu reserved for various utility functions

Right now we only have one utility fuction `save_model`

In [7]:
%%writefile going_modular/utils.py

"""
File containing various utility functions for pytorch model training
"""

from pathlib import Path
import torch

def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with `.pt` or `.pth"
    model_save_path = target_dir_path/ model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)

Overwriting going_modular/utils.py


In [13]:
%%writefile going_modular/train.py
"""
Trains a PyTorch image classification model using device agnostic code
"""

import os
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils
from timeit import default_timer as timer


# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001

# Setup directories
train_dir ="data/pizza_steak_sushi/train"
test_dir ="data/pizza_steak_sushi/test"

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else 'cpu'

# Create transforms
data_transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

# Create Dataloader and get class_names
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir, test_dir=test_dir, transform=data_transform, batch_size=BATCH_SIZE)

# Create a model
model = model_builder.TinyVGG(input_shape=3, hidden_units=HIDDEN_UNITS, output_shape= len(class_names)).to(device)

# Setup loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=LEARNING_RATE )

start_time = timer()

# Start training with help from engine.py
engine.train(model=model, train_dataloader=train_dataloader, test_dataloader=test_dataloader, loss_fn=loss_fn, optimizer=optimizer, epochs=NUM_EPOCHS, device=device)

end_time =timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model to file
utils.save_model(model= model, target_dir="models", model_name="05_going_model_script_mode_tiny_vgg.pth")


Overwriting going_modular/train.py


In [12]:
!python going_modular/train.py

^C
